In [1]:
import numpy as np
import pandas as pd
import sys
import os
import re
import matplotlib.pyplot as plt
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\Irfan Robbani\anaconda3\envs\new_general\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [67]:
df = pd.read_csv('Results\gpt4.1_results.csv')
df

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Irfan Robbani\AppData\Local\Temp\ipykernel_20252\2011548240.py:1: SyntaxWarning: invalid escape sequence '\g'
  df = pd.read_csv('Results\gpt4.1_results.csv')


,id,image_name,ocr_txt,full_response,output_extracted
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Output = Meme ini menyindir cowok-cowok zaman ...,Meme ini menyindir cowok-cowok zaman sekarang ...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Output = Meme ini menyampaikan pesan bahwa men...,Meme ini menyampaikan pesan bahwa menampilkan ...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,Meme ini membandingkan tingkat kematangan dagi...,"Rendang, makanan khas Indonesia (khususnya Min..."
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Output = Meme ini menggunakan permainan kata y...,Meme ini menggunakan permainan kata yang lucu....
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Pesan tersembunyi dari meme ini adalah sindira...,Pesan tersembunyi dari meme ini adalah sindira...
...,...,...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Pesan tersembunyi dari meme ini adalah sindira...,Pesan tersembunyi dari meme ini adalah sindira...
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Pesan tersembunyi dari meme ini adalah tentang...,Pesan tersembunyi dari meme ini adalah tentang...
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Output = Meme ini menyindir perbandingan antar...,Meme ini menyindir perbandingan antara Singapo...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Pesan tersembunyi dari meme ini adalah tentang...,Pesan tersembunyi dari meme ini adalah tentang...


In [68]:
df1 = pd.read_csv('Results\meta_maverick_results.csv')
df1

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Irfan Robbani\AppData\Local\Temp\ipykernel_20252\2085778579.py:1: SyntaxWarning: invalid escape sequence '\m'
  df1 = pd.read_csv('Results\meta_maverick_results.csv')


,id,image_name,ocr_txt,full_response,output_extracted
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Berdasarkan gambar dan teks meme yang disediak...,Berdasarkan gambar dan teks meme yang disediak...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Output = \nMeme ini ingin menyampaikan bahwa m...,Meme ini ingin menyampaikan bahwa memiliki aya...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,Meme ini menggunakan gambar potongan daging de...,Meme ini menggunakan gambar potongan daging de...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,"Kita ini orang yang tangguh dan kuat, sehingga...","Kita ini orang yang tangguh dan kuat, sehingga..."
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Output = Meme ini menyampaikan kritik terhadap...,Meme ini menyampaikan kritik terhadap pemiliha...
...,...,...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Orang Jawa sering kali merujuk pada penduduk y...,Meme ini menyindir bahwa orang Jawa adalah ora...
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Konteks dari meme ini adalah tentang perasaan ...,Konteks dari meme ini adalah tentang perasaan ...
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini membandingkan kinerja antara Singapor...,Meme ini membandingkan kinerja antara Singapor...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Output = \nMeme ini menggambarkan seseorang ya...,Meme ini menggambarkan seseorang yang tidak me...


In [69]:
df2 = pd.read_csv('Results\gemini_2.5_pro_results.csv')
df2

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Irfan Robbani\AppData\Local\Temp\ipykernel_20252\384074153.py:1: SyntaxWarning: invalid escape sequence '\g'
  df2 = pd.read_csv('Results\gemini_2.5_pro_results.csv')


,id,image_name,ocr_txt,full_response,output_extracted
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Meme ini menyindir para pria yang menginginkan...,Meme ini menyindir para pria yang menginginkan...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Meme ini menyampaikan pesan bahwa seseorang ya...,Meme ini menyampaikan pesan bahwa seseorang ya...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,Meme ini secara humoris membandingkan tingkat ...,Meme ini secara humoris membandingkan tingkat ...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Meme ini menyampaikan pesan kesombongan yang d...,Meme ini menyampaikan pesan kesombongan yang d...
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Meme ini adalah sebuah sindiran politik yang d...,Meme ini adalah sebuah sindiran politik yang d...
...,...,...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Meme ini menyindir fenomena sosial di mana ora...,Meme ini menyindir fenomena sosial di mana ora...
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Meme ini menggambarkan perasaan frustrasi dan ...,Meme ini menggambarkan perasaan frustrasi dan ...
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini menyajikan perbandingan ironis antara...,Meme ini menyajikan perbandingan ironis antara...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Meme ini menceritakan tentang seorang siswa be...,Meme ini menceritakan tentang seorang siswa be...


In [70]:
image = df['image_name'].tolist()
ocr = df['ocr_txt'].tolist()
gpt= df['output_extracted'].tolist()
meta = df1['output_extracted'].tolist()
gemini = df2['output_extracted'].tolist()

In [71]:
df_combine = pd.DataFrame({
    'image': image,
    'text': ocr,
    'gpt': gpt,
    'meta': meta,
    'gemini': gemini
})
df_combine

,image,text,gpt,meta,gemini
0,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Meme ini menyindir cowok-cowok zaman sekarang ...,Berdasarkan gambar dan teks meme yang disediak...,Meme ini menyindir para pria yang menginginkan...
1,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Meme ini menyampaikan pesan bahwa menampilkan ...,Meme ini ingin menyampaikan bahwa memiliki aya...,Meme ini menyampaikan pesan bahwa seseorang ya...
2,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,"Rendang, makanan khas Indonesia (khususnya Min...",Meme ini menggunakan gambar potongan daging de...,Meme ini secara humoris membandingkan tingkat ...
3,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Meme ini menggunakan permainan kata yang lucu....,"Kita ini orang yang tangguh dan kuat, sehingga...",Meme ini menyampaikan pesan kesombongan yang d...
4,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyampaikan kritik terhadap pemiliha...,Meme ini adalah sebuah sindiran politik yang d...
...,...,...,...,...,...
225,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyindir bahwa orang Jawa adalah ora...,Meme ini menyindir fenomena sosial di mana ora...
226,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Pesan tersembunyi dari meme ini adalah tentang...,Konteks dari meme ini adalah tentang perasaan ...,Meme ini menggambarkan perasaan frustrasi dan ...
227,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini menyindir perbandingan antara Singapo...,Meme ini membandingkan kinerja antara Singapor...,Meme ini menyajikan perbandingan ironis antara...
228,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Pesan tersembunyi dari meme ini adalah tentang...,Meme ini menggambarkan seseorang yang tidak me...,Meme ini menceritakan tentang seorang siswa be...


In [72]:
def clean_text(text):
    # Remove the word "meme" (case-insensitive)
    text = re.sub(r'\bmeme\b', ' ', text, flags=re.IGNORECASE)
    # Remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    # Remove numbers
    text = re.sub(r'\d+', ' ', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [73]:
df_combine['gpt_filtered'] = df_combine['gpt'].apply(clean_text)
df_combine['meta_filtered'] = df_combine['meta'].apply(clean_text)
df_combine['gemini_filtered'] = df_combine['gemini'].apply(clean_text)
df_combine 

,image,text,gpt,meta,gemini,gpt_filtered,meta_filtered,gemini_filtered
0,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Meme ini menyindir cowok-cowok zaman sekarang ...,Berdasarkan gambar dan teks meme yang disediak...,Meme ini menyindir para pria yang menginginkan...,ini menyindir cowok cowok zaman sekarang yang ...,Berdasarkan gambar dan teks yang disediakan pe...,ini menyindir para pria yang menginginkan pasa...
1,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Meme ini menyampaikan pesan bahwa menampilkan ...,Meme ini ingin menyampaikan bahwa memiliki aya...,Meme ini menyampaikan pesan bahwa seseorang ya...,ini menyampaikan pesan bahwa menampilkan ayat ...,ini ingin menyampaikan bahwa memiliki ayat suc...,ini menyampaikan pesan bahwa seseorang yang me...
2,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,"Rendang, makanan khas Indonesia (khususnya Min...",Meme ini menggunakan gambar potongan daging de...,Meme ini secara humoris membandingkan tingkat ...,Rendang makanan khas Indonesia khususnya Minan...,ini menggunakan gambar potongan daging dengan ...,ini secara humoris membandingkan tingkat kemat...
3,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Meme ini menggunakan permainan kata yang lucu....,"Kita ini orang yang tangguh dan kuat, sehingga...",Meme ini menyampaikan pesan kesombongan yang d...,ini menggunakan permainan kata yang lucu Kalim...,Kita ini orang yang tangguh dan kuat sehingga ...,ini menyampaikan pesan kesombongan yang dibalu...
4,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyampaikan kritik terhadap pemiliha...,Meme ini adalah sebuah sindiran politik yang d...,Pesan tersembunyi dari ini adalah sindiran ter...,ini menyampaikan kritik terhadap pemilihan ata...,ini adalah sebuah sindiran politik yang dituju...
...,...,...,...,...,...,...,...,...
225,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyindir bahwa orang Jawa adalah ora...,Meme ini menyindir fenomena sosial di mana ora...,Pesan tersembunyi dari ini adalah sindiran ter...,ini menyindir bahwa orang Jawa adalah orang ya...,ini menyindir fenomena sosial di mana orang or...
226,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Pesan tersembunyi dari meme ini adalah tentang...,Konteks dari meme ini adalah tentang perasaan ...,Meme ini menggambarkan perasaan frustrasi dan ...,Pesan tersembunyi dari ini adalah tentang peru...,Konteks dari ini adalah tentang perasaan tidak...,ini menggambarkan perasaan frustrasi dan ironi...
227,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini menyindir perbandingan antara Singapo...,Meme ini membandingkan kinerja antara Singapor...,Meme ini menyajikan perbandingan ironis antara...,ini menyindir perbandingan antara Singapore Ai...,ini membandingkan kinerja antara Singapore Air...,ini menyajikan perbandingan ironis antara mask...
228,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Pesan tersembunyi dari meme ini adalah tentang...,Meme ini menggambarkan seseorang yang tidak me...,Meme ini menceritakan tentang seorang siswa be...,Pesan tersembunyi dari ini adalah tentang sese...,ini menggambarkan seseorang yang tidak memakai...,ini menceritakan tentang seorang siswa bernama...


In [74]:
factory = StopWordRemoverFactory()
stopwords = set(factory.get_stop_words())

def remove_stopwords(text):
    tokens = text.split()
    filtered = [word for word in tokens if word.lower() not in stopwords]
    return ' '.join(filtered)

In [75]:
df_combine['gpt_filtered'] = df_combine['gpt_filtered'].apply(remove_stopwords)
df_combine['meta_filtered'] = df_combine['meta_filtered'].apply(remove_stopwords)
df_combine['gemini_filtered'] = df_combine['gemini_filtered'].apply(remove_stopwords)
df_combine

,image,text,gpt,meta,gemini,gpt_filtered,meta_filtered,gemini_filtered
0,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Meme ini menyindir cowok-cowok zaman sekarang ...,Berdasarkan gambar dan teks meme yang disediak...,Meme ini menyindir para pria yang menginginkan...,menyindir cowok cowok zaman sekarang punya pac...,Berdasarkan gambar teks disediakan pesan terse...,menyindir pria menginginkan pasangan cantik te...
1,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Meme ini menyampaikan pesan bahwa menampilkan ...,Meme ini ingin menyampaikan bahwa memiliki aya...,Meme ini menyampaikan pesan bahwa seseorang ya...,menyampaikan pesan menampilkan ayat suci bio m...,menyampaikan memiliki ayat suci profil media s...,menyampaikan pesan seseorang mencantumkan kuti...
2,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,"Rendang, makanan khas Indonesia (khususnya Min...",Meme ini menggunakan gambar potongan daging de...,Meme ini secara humoris membandingkan tingkat ...,Rendang makanan khas Indonesia khususnya Minan...,menggunakan gambar potongan daging tingkat kem...,humoris membandingkan tingkat kematangan steak...
3,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Meme ini menggunakan permainan kata yang lucu....,"Kita ini orang yang tangguh dan kuat, sehingga...",Meme ini menyampaikan pesan kesombongan yang d...,menggunakan permainan kata lucu Kalimat MAH OR...,orang tangguh kuat mudah dikalahkan dilawan Ka...,menyampaikan pesan kesombongan dibalut candaan...
4,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyampaikan kritik terhadap pemiliha...,Meme ini adalah sebuah sindiran politik yang d...,Pesan tersembunyi sindiran alasan alasan orang...,menyampaikan kritik pemilihan dukungan seseora...,sebuah sindiran politik ditujukan salah satu c...
...,...,...,...,...,...,...,...,...
225,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyindir bahwa orang Jawa adalah ora...,Meme ini menyindir fenomena sosial di mana ora...,Pesan tersembunyi sindiran orang orang sering ...,menyindir orang Jawa orang memiliki cakupan lu...,menyindir fenomena sosial mana orang orang tin...
226,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Pesan tersembunyi dari meme ini adalah tentang...,Konteks dari meme ini adalah tentang perasaan ...,Meme ini menggambarkan perasaan frustrasi dan ...,Pesan tersembunyi perubahan perspektif seiring...,Konteks perasaan nyaman kaget melihat biasa se...,menggambarkan perasaan frustrasi ironi mendala...
227,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini menyindir perbandingan antara Singapo...,Meme ini membandingkan kinerja antara Singapor...,Meme ini menyajikan perbandingan ironis antara...,menyindir perbandingan Singapore Airlines Garu...,membandingkan kinerja Singapore Airlines Garud...,menyajikan perbandingan ironis maskapai Singap...
228,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Pesan tersembunyi dari meme ini adalah tentang...,Meme ini menggambarkan seseorang yang tidak me...,Meme ini menceritakan tentang seorang siswa be...,Pesan tersembunyi seseorang dikerjai prank tem...,menggambarkan seseorang memakai seragam pramuk...,menceritakan seorang siswa bernama Yudi menjad...


In [76]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    return stemmer.stem(text)

In [77]:
df_combine['gpt_filtered'] = df_combine['gpt_filtered'].apply(stemming)
df_combine['meta_filtered'] = df_combine['meta_filtered'].apply(stemming)
df_combine['gemini_filtered'] = df_combine['gemini_filtered'].apply(stemming)

In [78]:
def remove_all_duplicate_words(text):
    seen = set()
    result = []
    for word in text.split():
        if word not in seen:
            seen.add(word)
            result.append(word)
    return ' '.join(result)


In [79]:
df_combine['gpt_filtered'] = df_combine['gpt_filtered'].apply(remove_all_duplicate_words)
df_combine['meta_filtered'] = df_combine['meta_filtered'].apply(remove_all_duplicate_words)
df_combine['gemini_filtered'] = df_combine['gemini_filtered'].apply(remove_all_duplicate_words)
df_combine

,image,text,gpt,meta,gemini,gpt_filtered,meta_filtered,gemini_filtered
0,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Meme ini menyindir cowok-cowok zaman sekarang ...,Berdasarkan gambar dan teks meme yang disediak...,Meme ini menyindir para pria yang menginginkan...,sindir cowok zaman sekarang punya pacar cantik...,dasar gambar teks sedia pesan sembunyi balik k...,sindir pria ingin pasang cantik awat mau kelua...
1,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Meme ini menyampaikan pesan bahwa menampilkan ...,Meme ini ingin menyampaikan bahwa memiliki aya...,Meme ini menyampaikan pesan bahwa seseorang ya...,sampai pesan tampil ayat suci bio media sosial...,sampai milik ayat suci profil media sosial bio...,sampai pesan orang cantum kutip kitab suci bio...
2,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,"Rendang, makanan khas Indonesia (khususnya Min...",Meme ini menggunakan gambar potongan daging de...,Meme ini secara humoris membandingkan tingkat ...,rendang makan khas indonesia khusus mang masak...,guna gambar potong daging tingkat matang beda ...,humoris banding tingkat matang steak ala barat...
3,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Meme ini menggunakan permainan kata yang lucu....,"Kita ini orang yang tangguh dan kuat, sehingga...",Meme ini menyampaikan pesan kesombongan yang d...,guna main kata lucu kalimat mah orang susah bi...,orang tangguh kuat mudah kalah lawan kalimat m...,sampai pesan sombong balut canda awal kalimat ...
4,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyampaikan kritik terhadap pemiliha...,Meme ini adalah sebuah sindiran politik yang d...,pesan sembunyi sindir alas orang pilih misal m...,sampai kritik pilih dukung orang mungkin konte...,buah sindir politik tuju salah satu calon pres...
...,...,...,...,...,...,...,...,...
225,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyindir bahwa orang Jawa adalah ora...,Meme ini menyindir fenomena sosial di mana ora...,pesan sembunyi sindir orang sering anggap band...,sindir orang jawa milik cakup luas pulau tanya...,sindir fenomena sosial mana orang tinggal jaka...
226,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Pesan tersembunyi dari meme ini adalah tentang...,Konteks dari meme ini adalah tentang perasaan ...,Meme ini menggambarkan perasaan frustrasi dan ...,pesan sembunyi ubah perspektif iring tambah us...,konteks asa nyaman kaget lihat biasa sesuai ha...,gambar asa frustrasi ironi dalam orang kena na...
227,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini menyindir perbandingan antara Singapo...,Meme ini membandingkan kinerja antara Singapor...,Meme ini menyajikan perbandingan ironis antara...,sindir banding singapore airlines garuda indon...,banding kerja singapore airlines garuda indone...,saji banding ironis maskapai singapore airline...
228,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Pesan tersembunyi dari meme ini adalah tentang...,Meme ini menggambarkan seseorang yang tidak me...,Meme ini menceritakan tentang seorang siswa be...,pesan sembunyi orang kerja prank teman chat yu...,gambar orang pakai seragam pramuka acu ikut at...,cerita orang siswa nama yudi jadi korban iseng...


In [80]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def compute_similarities(row):
    messages = [row['gpt_filtered'], row['meta_filtered'], row['gemini_filtered']]
    embeddings = model.encode(messages)
    sim_matrix = cosine_similarity(embeddings)
    sim_gpt_meta = sim_matrix[0, 1]
    sim_gpt_gemini = sim_matrix[0, 2]
    sim_meta_gemini = sim_matrix[1, 2]
    avg_sim = np.mean([sim_gpt_meta, sim_gpt_gemini, sim_meta_gemini])
    return pd.Series([sim_gpt_meta, sim_gpt_gemini, sim_meta_gemini, avg_sim],
                     index=["sim_gpt_meta", "sim_gpt_gemini", "sim_meta_gemini", "avg_similarity"])

df_sim = df_combine.apply(compute_similarities, axis=1)
df_combine = pd.concat([df_combine, df_sim], axis=1)

c:\Users\Irfan Robbani\anaconda3\envs\new_general\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\Irfan Robbani\anaconda3\envs\new_general\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\Irfan Robbani\anaconda3\envs\new_general\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\Irfan Robbani\anaconda3\envs\new_general\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 

In [81]:
df_combine

,image,text,gpt,meta,gemini,gpt_filtered,meta_filtered,gemini_filtered,sim_gpt_meta,sim_gpt_gemini,sim_meta_gemini,avg_similarity
0,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Meme ini menyindir cowok-cowok zaman sekarang ...,Berdasarkan gambar dan teks meme yang disediak...,Meme ini menyindir para pria yang menginginkan...,sindir cowok zaman sekarang punya pacar cantik...,dasar gambar teks sedia pesan sembunyi balik k...,sindir pria ingin pasang cantik awat mau kelua...,0.707763,0.713677,0.789050,0.736830
1,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Meme ini menyampaikan pesan bahwa menampilkan ...,Meme ini ingin menyampaikan bahwa memiliki aya...,Meme ini menyampaikan pesan bahwa seseorang ya...,sampai pesan tampil ayat suci bio media sosial...,sampai milik ayat suci profil media sosial bio...,sampai pesan orang cantum kutip kitab suci bio...,0.846443,0.905368,0.860756,0.870856
2,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,"Rendang, makanan khas Indonesia (khususnya Min...",Meme ini menggunakan gambar potongan daging de...,Meme ini secara humoris membandingkan tingkat ...,rendang makan khas indonesia khusus mang masak...,guna gambar potong daging tingkat matang beda ...,humoris banding tingkat matang steak ala barat...,0.729216,0.885666,0.745329,0.786737
3,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Meme ini menggunakan permainan kata yang lucu....,"Kita ini orang yang tangguh dan kuat, sehingga...",Meme ini menyampaikan pesan kesombongan yang d...,guna main kata lucu kalimat mah orang susah bi...,orang tangguh kuat mudah kalah lawan kalimat m...,sampai pesan sombong balut canda awal kalimat ...,0.763913,0.804526,0.724823,0.764421
4,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyampaikan kritik terhadap pemiliha...,Meme ini adalah sebuah sindiran politik yang d...,pesan sembunyi sindir alas orang pilih misal m...,sampai kritik pilih dukung orang mungkin konte...,buah sindir politik tuju salah satu calon pres...,0.829906,0.549386,0.620072,0.666455
...,...,...,...,...,...,...,...,...,...,...,...,...
225,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyindir bahwa orang Jawa adalah ora...,Meme ini menyindir fenomena sosial di mana ora...,pesan sembunyi sindir orang sering anggap band...,sindir orang jawa milik cakup luas pulau tanya...,sindir fenomena sosial mana orang tinggal jaka...,0.756237,0.844710,0.708458,0.769801
226,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Pesan tersembunyi dari meme ini adalah tentang...,Konteks dari meme ini adalah tentang perasaan ...,Meme ini menggambarkan perasaan frustrasi dan ...,pesan sembunyi ubah perspektif iring tambah us...,konteks asa nyaman kaget lihat biasa sesuai ha...,gambar asa frustrasi ironi dalam orang kena na...,0.857113,0.828321,0.720527,0.801987
227,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini menyindir perbandingan antara Singapo...,Meme ini membandingkan kinerja antara Singapor...,Meme ini menyajikan perbandingan ironis antara...,sindir banding singapore airlines garuda indon...,banding kerja singapore airlines garuda indone...,saji banding ironis maskapai singapore airline...,0.931439,0.914472,0.926283,0.924065
228,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Pesan tersembunyi dari meme ini adalah tentang...,Meme ini menggambarkan seseorang yang tidak me...,Meme ini menceritakan tentang seorang siswa be...,pesan sembunyi orang kerja prank teman chat yu...,gambar orang pakai seragam pramuka acu ikut at...,cerita orang siswa nama yudi jadi korban iseng...,0.523057,0.786242,0.435058,0.581452


In [82]:
print("Average similarity across all rows:", df_combine["avg_similarity"].mean())
print("Average similarity between GPT4.1 and Meta Maverick:", df_combine["sim_gpt_meta"].mean())
print("Average similarity between GPT4.1 and Gemini:", df_combine["sim_gpt_gemini"].mean())
print("Average similarity between Meta Maverick and gemini:", df_combine["sim_meta_gemini"].mean())

Average similarity across all rows: 0.7041769
Average similarity between GPT4.1 and Meta Maverick: 0.687799
Average similarity between GPT4.1 and Gemini: 0.76208454
Average similarity between Meta Maverick and gemini: 0.6626471


In [83]:
# For image filenames
image_list = df_combine[df_combine["avg_similarity"] < 0.5]["image"].tolist()
print(image_list)


['Meme_Data_8.jpg', 'Meme_Data_24.jpg', 'Meme_Data_28.jpg', 'Meme_Data_35.jpg', 'Meme_Data_46.jpg', 'Meme_Data_90.jpg', 'Meme_Data_107.jpg', 'Meme_Data_123.jpg', 'Meme_Data_184.jpg']


In [84]:
df_combine.to_csv('Results/sentence_embed_results_filtered.csv', index=False)

In [51]:
df_combine['gpt_filtered'] = df_combine['gpt_filtered'].apply(lambda x: x.split())
df_combine['meta_filtered'] = df_combine['meta_filtered'].apply(lambda x: x.split())
df_combine['gemini_filtered'] = df_combine['gemini_filtered'].apply(lambda x: x.split())
df_combine

,image,text,gpt,meta,gemini,gpt_filtered,meta_filtered,gemini_filtered
0,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Meme ini menyindir cowok-cowok zaman sekarang ...,Berdasarkan gambar dan teks meme yang disediak...,Meme ini menyindir para pria yang menginginkan...,"[sindir, cowok, zaman, sekarang, punya, pacar,...","[dasar, gambar, teks, sedia, pesan, sembunyi, ...","[sindir, pria, ingin, pasang, cantik, awat, ma..."
1,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Meme ini menyampaikan pesan bahwa menampilkan ...,Meme ini ingin menyampaikan bahwa memiliki aya...,Meme ini menyampaikan pesan bahwa seseorang ya...,"[sampai, pesan, tampil, ayat, suci, bio, media...","[sampai, milik, ayat, suci, profil, media, sos...","[sampai, pesan, orang, cantum, kutip, kitab, s..."
2,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,"Rendang, makanan khas Indonesia (khususnya Min...",Meme ini menggunakan gambar potongan daging de...,Meme ini secara humoris membandingkan tingkat ...,"[rendang, makan, khas, indonesia, khusus, mang...","[guna, gambar, potong, daging, tingkat, matang...","[humoris, banding, tingkat, matang, steak, ala..."
3,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Meme ini menggunakan permainan kata yang lucu....,"Kita ini orang yang tangguh dan kuat, sehingga...",Meme ini menyampaikan pesan kesombongan yang d...,"[guna, main, kata, lucu, kalimat, mah, orang, ...","[orang, tangguh, kuat, mudah, kalah, lawan, ka...","[sampai, pesan, sombong, balut, canda, awal, k..."
4,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyampaikan kritik terhadap pemiliha...,Meme ini adalah sebuah sindiran politik yang d...,"[pesan, sembunyi, sindir, alas, orang, pilih, ...","[sampai, kritik, pilih, dukung, orang, mungkin...","[buah, sindir, politik, tuju, salah, satu, cal..."
...,...,...,...,...,...,...,...,...
225,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Pesan tersembunyi dari meme ini adalah sindira...,Meme ini menyindir bahwa orang Jawa adalah ora...,Meme ini menyindir fenomena sosial di mana ora...,"[pesan, sembunyi, sindir, orang, sering, angga...","[sindir, orang, jawa, milik, cakup, luas, pula...","[sindir, fenomena, sosial, mana, orang, tingga..."
226,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Pesan tersembunyi dari meme ini adalah tentang...,Konteks dari meme ini adalah tentang perasaan ...,Meme ini menggambarkan perasaan frustrasi dan ...,"[pesan, sembunyi, ubah, perspektif, iring, tam...","[konteks, asa, nyaman, kaget, lihat, biasa, se...","[gambar, asa, frustrasi, ironi, dalam, orang, ..."
227,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini menyindir perbandingan antara Singapo...,Meme ini membandingkan kinerja antara Singapor...,Meme ini menyajikan perbandingan ironis antara...,"[sindir, banding, singapore, airlines, garuda,...","[banding, kerja, singapore, airlines, garuda, ...","[saji, banding, ironis, maskapai, singapore, a..."
228,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Pesan tersembunyi dari meme ini adalah tentang...,Meme ini menggambarkan seseorang yang tidak me...,Meme ini menceritakan tentang seorang siswa be...,"[pesan, sembunyi, orang, kerja, prank, teman, ...","[gambar, orang, pakai, seragam, pramuka, acu, ...","[cerita, orang, siswa, nama, yudi, jadi, korba..."
